In [ ]:
!pip install ipywidgets >> /dev/null
!pip install pyaudio >> /dev/null

In [ ]:
import json
import pyaudio
import subprocess
from queue import Queue
from threading import Thread
import ipywidgets as widgets
from vosk import Model, KaldiRecognizer

In [ ]:
messages = Queue()
recordings = Queue()

In [ ]:
from IPython.display import display
record_button = widgets.Button(
    description="Record",
    disavbled=False,
    button_style="success",
    icon="microphone"
)

stop_button = widgets.Button(
    description="Stop",
    disavbled=False,
    button_style="warning",
    icon="stop"
)


output = widgets.Output()

def start_recording(data):
    messages.put(True)

    with output:
        display("Starting...")
        record = Thread(target=record_microphone)
        record.start()

        transcribe  = Thread(target=speech_recognition, args=(output,))
        transcribe.start()
        

def stop_recording(data):
    with output:
        messages.get()
        display("Stopped.")


record_button.on_click(start_recording)
stop_button.on_click(stop_recording)
display(record_button, stop_button, output)

In [ ]:
p = pyaudio.PyAudio()
for i in range(p.get_device_count()):
    print(p.get_device_info_by_index(i))

p.terminate()

In [ ]:
CHANNELS = 2
FRAME_RATE = 16000
RECORD_SECONDs = 1
AUDIO_FORMAT = pyaudio.paInt16

def record_microphone(chunk=1024):
    p = pyaudio.PyAudio()

    stream = p.open(format=AUDIO_FORMAT,
                    channels=CHANNELS,
                    rate=FRAME_RATE,
                    input=True,
                    input_device_index=9,
                    frames_per_buffer=chunk)
    
    frames = []

    while not messages.empty():
        data = stream.read(chunk)
        frames.append(data)

        if len(frames) >= (FRAME_RATE * RECORD_SECONDs) / chunk:
            recordings.put(frames.copy())
            frames = []

    stream.stop_stream()
    stream.close()
    p.terminate()

In [ ]:
model = Model(model_name="model.pth")
rec = KaldiRecognizer(model, FRAME_RATE)
rec.SetWords(True)

In [ ]:
def speech_recognition(output):
    from time import sleep
    while not messages.empty():
        frames = recordings.get()

        rec.AcceptWaveform(b''.join(frames))
        result = rec.Result()
        text = json.loads(result)["text"]
        output.append_stdout(text)
